In [ ]:
import os 

In [ ]:
from llama_index.legacy import GPTListIndex, SlackReader
slack_token = os.getenv("SLACK_TOKEN")
channel_ids = ["hpc"]
documents = SlackReader(slack_token=slack_token).load_data(channel_ids=channel_ids)
index = GPTListIndex.from_documents(documents)

In [ ]:
from slack_sdk import WebClient
client = WebClient(token=slack_token)
res = client.api_test()
if not res["ok"]:
    raise ValueError(f"Error initializing Slack API: {res['error']}")

In [ ]:
request = client.api_call("users.list")
request['members'][3]

In [ ]:
user_map = {}
request = client.api_call("users.list")
if request['ok']:
    for item in request['members']:
        id = item['id']
        username = item['name']
        name = item['profile']['real_name']
        user_map[id] = name

print(f"{len(user_map)} users")

In [ ]:

from typing import List, Optional
import time
import logging
logger = logging.getLogger()

def _read_message(channel_id: str, message_ts: str) -> str:
    from slack_sdk.errors import SlackApiError

    """Read a message."""

    messages_text: List[str] = []
    next_cursor = None
    while True:
        try:
            # https://slack.com/api/conversations.replies
            # List all replies to a message, including the message itself.
            result = client.conversations_replies(
                channel=channel_id, ts=message_ts, cursor=next_cursor
            )
            messages = result["messages"]
            messages_text.extend(message["text"] for message in messages)
            if not result["has_more"]:
                break

            next_cursor = result["response_metadata"]["next_cursor"]
        except SlackApiError as e:
            if e.response["error"] == "ratelimited":
                logger.error(
                    "Rate limit error reached, sleeping for: {} seconds".format(
                        e.response.headers["retry-after"]
                    )
                )
                time.sleep(int(e.response.headers["retry-after"]))
            else:
                logger.error("Error parsing conversation replies: {}".format(e))

    return "\n\n".join(messages_text)

def _read_channel(channel_id: str, reverse_chronological: bool) -> str:
    from slack_sdk.errors import SlackApiError

    """Read a channel."""

    result_messages: List[str] = []
    next_cursor = None
    while True:
        try:
            # Call the conversations.history method using the WebClient
            # conversations.history returns the first 100 messages by default
            # These results are paginated,
            # see: https://api.slack.com/methods/conversations.history$pagination
            conversations_history_kwargs = {
                "channel": channel_id,
                "cursor": next_cursor,
            }
            result = client.conversations_history(
                **conversations_history_kwargs  # type: ignore
            )
            conversation_history = result["messages"]
            # Print results
            logger.info(
                "{} messages found in {}".format(
                    len(conversation_history), channel_id
                )
            )
            result_messages.extend(
                _read_message(channel_id, message["ts"])
                for message in conversation_history
            )
            if not result["has_more"]:
                break
            next_cursor = result["response_metadata"]["next_cursor"]

        except SlackApiError as e:
            if e.response["error"] == "ratelimited":
                logger.error(
                    "Rate limit error reached, sleeping for: {} seconds".format(
                        e.response.headers["retry-after"]
                    )
                )
                time.sleep(int(e.response.headers["retry-after"]))
            else:
                logger.error("Error parsing conversation replies: {}".format(e))

    return (
        "\n\n".join(result_messages)
        if reverse_chronological
        else "\n\n".join(result_messages[::-1])
    )




In [ ]:
channel_id = "C019ZPNLLA0" # workflows
channel_content = _read_channel(
    channel_id, reverse_chronological=True
)


In [ ]:
print(channel_content)

In [ ]:

channel_id = "C057Z7J7F29"
message_ts = "1684259866.667699"

res = client.chat_getPermalink(channel=channel_id, message_ts=message_ts)
if res['ok']:
    print(res['permalink'])
